# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [70]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import os
import csv

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [71]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

609

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it's being processed (with the city number and city name).


In [72]:
api = weather_api_key
# Build query URL
url = "http://api.openweathermap.org/data/2.5/weather?"

def APIRequest(cityNumber):#function for making the API call to OpenWeatherMap.org and returning the data
    query_url = url + "appid=" + api + "&q=" + cities[cityNumber-1]
    responseJson = requests.get(query_url).json() #making the API request and grabbing the JSON from the response

    if ("'message': 'city not found'" in str(responseJson)): #checking response JSON for the string of text containing the message 'city not found'
        full_City_Data.append(" ")#if the city is not found, append a null value to the lists to keep the index numbers correct
        city_City_Data.append(" ")#list for city names
        lat_City_Data.append(" ")#list for city latitude
        lng_City_Data.append(" ")#list for city longitude
        max_Temp_City_Data.append(" ")#list for city maximum temperature
        humidity_City_Data.append(" ")#list for city humidity
        cloudiness_City_Data.append(" ")#list for city cloudiness amount
        wind_Speed_City_Data.append(" ")#list for city wind speed amount
        country_City_Data.append(" ")#list for which country a city is in
        date_City_Data.append(" ")#list for the datetime information stored in unix UTC format
        print("City not found. Skipping...")
    else:
        full_City_Data.append(responseJson) #if the city is found, do append the information to the lists
        city_City_Data.append(responseJson["name"])
        lat_City_Data.append(responseJson["coord"]["lat"])
        lng_City_Data.append(responseJson["coord"]["lat"])
        max_Temp_City_Data.append(responseJson["main"]["temp_max"])
        humidity_City_Data.append(responseJson["main"]["humidity"])
        cloudiness_City_Data.append(responseJson["clouds"]["all"])
        wind_Speed_City_Data.append(responseJson["wind"]["speed"])
        country_City_Data.append(responseJson["sys"]["country"])
        date_City_Data.append(responseJson["dt"])

#if length of cities is exactly divisible by 50 then get the length. else we need to add an extra batch to account for the remainder
remainder = 0
if (len(cities) % 50 == 0):
        totalBatches = int(len(cities) / 50) #doing batches of 50 API calls at once; finding the total number of batches I will be making
else:
    totalBatches = int((len(cities) / 50)+1)
    remainder = int(len(cities) % 50)
    
batchCounter = 1
requestsPerBatch = 50
requestCounter = 1
cityNumber = 1
#creating empty lists to use to append each piece of the weather data response to
full_City_Data, city_City_Data, lat_City_Data, lng_City_Data, max_Temp_City_Data, humidity_City_Data, cloudiness_City_Data, wind_Speed_City_Data, country_City_Data, date_City_Data = [], [], [], [], [], [], [], [], [], []

print("Beginning Data Retrieval\n-----------------------------")
while (batchCounter <= totalBatches): 
    print(f"Starting batch {batchCounter} of {totalBatches}.")
    while ((batchCounter != totalBatches) & (requestCounter <= requestsPerBatch)): #if we're not on the final batch, run this code
        cityNumber = (50 * (batchCounter-1)) + requestCounter #calculating city number        
        APIRequest(cityNumber)
        if (city_City_Data[cityNumber-1] != " "):#if any of lists at this element are blank, skip printing the name and move onto the next city
            print(f"Request {requestCounter} of batch {batchCounter}. City number: {cityNumber} City name: {city_City_Data[cityNumber-1]}.")
        requestCounter +=1
    while ((batchCounter == totalBatches) & (requestCounter <= remainder)): #if we are on the final batch, run this code
        cityNumber = (50 * (batchCounter-1)) + requestCounter #calculating city number
        APIRequest(cityNumber)
        if (city_City_Data[cityNumber-1] != " "):#if any of lists at this element are blank, skip printing the name and move onto the next city
            print(f"Request {requestCounter} of batch {batchCounter}. City number: {cityNumber} City name: {city_City_Data[cityNumber-1]}.")
        requestCounter +=1        
    batchCounter +=1
    requestCounter = 1 #resetting requestCounter so it will hit the inner while loop again in the next iteration of the outer while loop
    
    #break ##break out of out while loop so i don't make so many API calls during testing##
print("-----------------------------\nData Retrieval Complete\n----------------------------") 

Beginning Data Retrieval
-----------------------------
Starting batch 1 of 13.
Request 1 of batch 1. City number: 1 City name: Hermanus.
Request 2 of batch 1. City number: 2 City name: Yaan.
Request 3 of batch 1. City number: 3 City name: Bredasdorp.
Request 4 of batch 1. City number: 4 City name: Poum.
Request 5 of batch 1. City number: 5 City name: Nikolskoye.
Request 6 of batch 1. City number: 6 City name: Adrar.
Request 7 of batch 1. City number: 7 City name: Atuona.
Request 8 of batch 1. City number: 8 City name: Jutaí.
Request 9 of batch 1. City number: 9 City name: Cologne.
Request 10 of batch 1. City number: 10 City name: Jamestown.
Request 11 of batch 1. City number: 11 City name: Rawson.
Request 12 of batch 1. City number: 12 City name: Imbituba.
Request 13 of batch 1. City number: 13 City name: Drăgeşti.
City not found. Skipping...
Request 15 of batch 1. City number: 15 City name: Albany.
City not found. Skipping...
Request 17 of batch 1. City number: 17 City name: Marzuq.
R

Request 40 of batch 3. City number: 140 City name: Balabac.
Request 41 of batch 3. City number: 141 City name: Cabo San Lucas.
Request 42 of batch 3. City number: 142 City name: Osečina.
Request 43 of batch 3. City number: 143 City name: Kargil.
Request 44 of batch 3. City number: 144 City name: Taclobo.
Request 45 of batch 3. City number: 145 City name: Dombarovskiy.
Request 46 of batch 3. City number: 146 City name: Gunjur.
Request 47 of batch 3. City number: 147 City name: Yumen.
Request 48 of batch 3. City number: 148 City name: Saint-Joseph.
Request 49 of batch 3. City number: 149 City name: Maceió.
Request 50 of batch 3. City number: 150 City name: Aksu.
Starting batch 4 of 13.
Request 1 of batch 4. City number: 151 City name: Hobart.
Request 2 of batch 4. City number: 152 City name: Dudinka.
Request 3 of batch 4. City number: 153 City name: Virden.
Request 4 of batch 4. City number: 154 City name: Gijang.
Request 5 of batch 4. City number: 155 City name: Murgab.
Request 6 of bat

Request 32 of batch 6. City number: 282 City name: Pevek.
City not found. Skipping...
Request 34 of batch 6. City number: 284 City name: Pilar.
Request 35 of batch 6. City number: 285 City name: Seabra.
Request 36 of batch 6. City number: 286 City name: Beringovskiy.
Request 37 of batch 6. City number: 287 City name: Sarakhs.
Request 38 of batch 6. City number: 288 City name: Pasichna.
Request 39 of batch 6. City number: 289 City name: Kirkuk.
Request 40 of batch 6. City number: 290 City name: Chabahar.
Request 41 of batch 6. City number: 291 City name: Dunedin.
Request 42 of batch 6. City number: 292 City name: Oktyabr'skoye.
Request 43 of batch 6. City number: 293 City name: Yulara.
Request 44 of batch 6. City number: 294 City name: Barranca.
Request 45 of batch 6. City number: 295 City name: Petrovac.
Request 46 of batch 6. City number: 296 City name: Meulaboh.
Request 47 of batch 6. City number: 297 City name: Baykit.
Request 48 of batch 6. City number: 298 City name: Nabire.
Reque

Request 24 of batch 9. City number: 424 City name: Lima.
Request 25 of batch 9. City number: 425 City name: Tiksi.
Request 26 of batch 9. City number: 426 City name: Riyadh.
Request 27 of batch 9. City number: 427 City name: Springdale.
Request 28 of batch 9. City number: 428 City name: Vila.
Request 29 of batch 9. City number: 429 City name: Giohar.
Request 30 of batch 9. City number: 430 City name: Ribeira Grande.
Request 31 of batch 9. City number: 431 City name: Lashio.
Request 32 of batch 9. City number: 432 City name: Mahon.
City not found. Skipping...
Request 34 of batch 9. City number: 434 City name: Berlevåg.
Request 35 of batch 9. City number: 435 City name: Sitka.
Request 36 of batch 9. City number: 436 City name: Shaoguan.
Request 37 of batch 9. City number: 437 City name: Ketchikan.
Request 38 of batch 9. City number: 438 City name: Mangan.
Request 39 of batch 9. City number: 439 City name: Kieta.
Request 40 of batch 9. City number: 440 City name: Bengkulu.
Request 41 of b

Request 10 of batch 12. City number: 560 City name: Salamiyah.
Request 11 of batch 12. City number: 561 City name: Charters Towers.
Request 12 of batch 12. City number: 562 City name: Billings Metropolitan Area.
Request 13 of batch 12. City number: 563 City name: Kampong Thom.
Request 14 of batch 12. City number: 564 City name: Megion.
Request 15 of batch 12. City number: 565 City name: Pozo Colorado.
Request 16 of batch 12. City number: 566 City name: Albeşti.
Request 17 of batch 12. City number: 567 City name: Ginir.
Request 18 of batch 12. City number: 568 City name: Kushmurun.
Request 19 of batch 12. City number: 569 City name: Honiara.
Request 20 of batch 12. City number: 570 City name: Lugovoy.
Request 21 of batch 12. City number: 571 City name: Bandarbeyla.
Request 22 of batch 12. City number: 572 City name: Chebsara.
Request 23 of batch 12. City number: 573 City name: Qiongshan.
Request 24 of batch 12. City number: 574 City name: Hofn.
City not found. Skipping...
Request 26 of 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [77]:
#use each individual list as a column in the data frame
weather_df = pd.DataFrame(data={"City": city_City_Data, "Lat": lat_City_Data, "Lng": lng_City_Data, "Max Temp": max_Temp_City_Data, "Humidity": humidity_City_Data, "Cloudiness": cloudiness_City_Data, "Wind Speed": wind_Speed_City_Data, "Country": country_City_Data, "Date": date_City_Data})

output_path = os.path.join("Output", "output.csv")
weather_df.to_csv(output_path, sep=',',index=False)#write the data frame to a CSV

weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hermanus,-34.42,-34.42,291.48,78,3,0.45,ZA,1607925139
1,Yaan,7.38,7.38,297.47,84,13,2.37,NG,1607925139
2,Bredasdorp,-34.53,-34.53,289.15,77,2,3.1,ZA,1607925139
3,Poum,-20.23,-20.23,300.54,80,99,10.69,NC,1607925139
4,Nikolskoye,59.7,59.7,269.82,85,90,1,RU,1607925139
...,...,...,...,...,...,...,...,...,...
604,Sāmbhar,26.92,26.92,290.15,45,20,1.5,IN,1607925223
605,Takoradze,4.88,4.88,298.71,87,22,2.65,GH,1607925223
606,Santa Vitória do Palmar,-33.52,-33.52,287.3,87,48,3.38,BR,1607925223
607,Aykhal,66,66,240.15,67,0,1,RU,1607925223


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [74]:
#  Get the indices of cities that have humidity over 100%.


In [75]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression